In [97]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm.notebook import tqdm

## Подготовка среды и функций запросов

1. Запрашиваем access token по адресу: https://vkhost.github.io/
2. Получаем ответ и сохраняем в файле token.txt

In [107]:
with open("C:/Jupyter_Notebooks/token.txt") as f:
  token = f.read()

Создадим для удобства функции

In [45]:
def groupsGet(user_id):
    version = '5.199'
    url = 'https://api.vk.com/method/groups.get' + '?' +'user_id=' + str(user_id) + '&v=' + version + '&access_token=' +  token
    response = requests.get(url)
    return response.json().get('response')

In [46]:
def friendsGet(user_id):
    version = '5.199'
    url = 'https://api.vk.com/method/friends.get?user_id=' + str(user_id) + '&v=' + version + '&access_token=' +  token
    response = requests.get(url)
    return response.json().get('response')

In [144]:
def wallGet(owner_id):
    version = '5.199'
    url = 'https://api.vk.com/method/wall.get?owner_id=' + str(owner_id) + '&count=100&v=' + version + '&access_token=' +  token
    response = requests.get(url)
    return response.json().get('response')

In [48]:
def isLiked(user_id, owner_id, item_id):
    version = '5.199'
    url = 'https://api.vk.com/method/likes.isLiked?user_id=' + str(user_id) + '&type=post&owner_id=' + str(owner_id) + '&item_id=' + str(item_id) + '&v=' + version + '&access_token=' +  token
    response = requests.get(url)
    return response.json().get('response')

# Анализ поведения друзей ВКонтакте
1. Выгрузить посты из лент четырех друзей и меня. Записать результат в Pandas DataFrame и в CSV файл
2. Получить данные об их лайках и комментах. Записать результат в Pandas DataFrame и в CSV файл
3. Построить Superset-дашборд с оценкой среднего CTR (лайк, коммент) по постам в топ 10 групп моих топ 4 друзей и меня
4. Построить Superset-дашборд с оценкой среднего CTR (лайк, коммент) по друзьям и мне

In [139]:
user_ids = ['303943594', '413980874', '519552111', '519556029']

In [140]:
names = ['Алина', 'Арсений', 'Татьяна', 'Олег']

## 1. Выгрузка постов

In [145]:
for i, user_id in enumerate(user_ids):
    print(names[i])
    feed = {
        'page_id': [],
        'post_id': [],
        'text': [],
        'likes': [],
        'comments': [],
        'views': []
    }
    
    groups = groupsGet(user_id)['items']
    for group in tqdm(groups):
        time.sleep(0.4)
        wall = wallGet('-' + str(group))
        if wall != None:
            wall = wall['items']
            feed['page_id'].extend(['-' + str(group) for post in wall])
            feed['post_id'].extend([post.get('id') for post in wall])
            feed['text'].extend([post.get('text') for post in wall])
            feed['likes'].extend([post.get('likes').get('count') if post.get('likes') != None else None for post in wall])
            feed['comments'].extend([post.get('comments').get('count') if post.get('comments') != None else None for post in wall])
            feed['views'].extend([post.get('views').get('count') if post.get('views') != None else None for post in wall])
    
    friends = friendsGet(str(user_id))['items']
    for friend in tqdm(friends):
        time.sleep(0.4)
        wall = wallGet(friend)
        if wall != None:
            wall = wall['items']
            feed['page_id'].extend([str(friend) for post in wall])
            feed['post_id'].extend([post.get('id') for post in wall])
            feed['text'].extend([post.get('text') for post in wall])
            feed['likes'].extend([post.get('likes').get('count') if post.get('likes') != None else None for post in wall])
            feed['comments'].extend([post.get('comments').get('count') if post.get('comments') != None else None for post in wall])
            feed['views'].extend([post.get('views').get('count') if post.get('views') != None else None for post in wall])

    df = pd.DataFrame(feed)
    df.to_csv('Лента_' + names[i])

Алина


C:\Users\zinov\AppData\Local\Temp\ipykernel_15724\790768899.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(groups):


  0%|          | 0/34 [00:00<?, ?it/s]

C:\Users\zinov\AppData\Local\Temp\ipykernel_15724\790768899.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for friend in tqdm_notebook(friends):


  0%|          | 0/64 [00:00<?, ?it/s]

Арсений


  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/144 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 2. Проверка наличия лайка

In [153]:
def checkLikes(id, csv_path):
    df = pd.read_csv(csv_path, index_col=0)
    df['isLiked'] = 0
    for i in tqdm(range(3801, len(df))):
        time.sleep(0.4)
        temp = isLiked(id, df['page_id'][i], df['post_id'][i])
        df.loc[i, 'isLiked'] = temp['liked'] if temp != None else None
        if temp == None:
            print('ВК ответил ошибкой')
        if i % 100 == 0:
            df.to_csv(csv_path + '_3')
            print('Сохранено на:', i)

In [156]:
for i, user_id in enumerate(user_ids):
    checkLikes(user_id, "C:/Users/zinov/Jupyter_Notebook/Лента_" + names[i] + '_с_лайками_с_лайками_2')

  0%|          | 0/1068 [00:00<?, ?it/s]

Сохранено на: 3900
Сохранено на: 4000
Сохранено на: 4100
Сохранено на: 4200
Сохранено на: 4300
Сохранено на: 4400
Сохранено на: 4500
Сохранено на: 4600
Сохранено на: 4700
Сохранено на: 4800


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/zinov/Jupyter_Notebook/Лента_Арсений_с_лайками_с_лайками_2'